In [2]:
import pandas as pd
import json
import cloudscraper
import certifi
from parsel import Selector
from bs4 import BeautifulSoup
import pprint

In [3]:
scraper = cloudscraper.create_scraper()
r = scraper.get('https://www.olx.com.br/imoveis/venda/apartamentos/estado-pe/grande-recife/recife?o=1', verify=certifi.where())

soup = BeautifulSoup(r.text, 'html.parser')
script_data = soup.find('script', {'id': '__NEXT_DATA__'}).string

html = json.loads(script_data)

dados = html.get('props', {}).get('pageProps', {}).get('ads', [])

In [4]:
pprint.pprint(dados[0])

{'accountActivityStatus': {'isOnline': False},
 'category': 'Apartamentos',
 'date': 1724531466,
 'fixedOnTop': True,
 'hasRealEstateHighlight': False,
 'imageCount': 16,
 'images': [{'original': 'https://img.olx.com.br/images/52/525330567528087.jpg',
             'originalAlt': 'Candeias Prince - 2102 B - Proprietário',
             'originalWebP': 'https://img.olx.com.br/images/52/525330567528087.webp',
             'thumbnail': 'https://img.olx.com.br/thumbs/52/525330567528087.jpg'},
            {'original': 'https://img.olx.com.br/images/51/514325204774933.jpg',
             'originalAlt': 'Candeias Prince - 2102 B - Proprietário - Foto 2',
             'originalWebP': 'https://img.olx.com.br/images/51/514325204774933.webp',
             'thumbnail': 'https://img.olx.com.br/thumbs/51/514325204774933.jpg'},
            {'original': 'https://img.olx.com.br/images/46/467367804295138.jpg',
             'originalAlt': 'Candeias Prince - 2102 B - Proprietário - Foto 3',
             'ori

In [5]:
# Listar os atributos de cada anuncio
atributos = dados[0].keys()
print(f"Atributos disponíveis no anúncio: {list(atributos)}")

Atributos disponíveis no anúncio: ['subject', 'title', 'price', 'listId', 'lastBumpAgeSecs', 'oldPrice', 'professionalAd', 'isFeatured', 'listingCategoryId', 'images', 'videoCount', 'isChatEnabled', 'fixedOnTop', 'url', 'thumbnail', 'date', 'imageCount', 'location', 'locationDetails', 'category', 'searchCategoryLevelZero', 'searchCategoryLevelOne', 'properties', 'accountActivityStatus', 'position', 'olxPay', 'olxPayBadgeEnabled', 'olxDelivery', 'olxDeliveryBadgeEnabled', 'installments', 'vehicleReport', 'vehicleTags', 'vehicleHasInspectionReport', 'vehiclePills', 'isFavorited', 'hasRealEstateHighlight', 'trackingSpecificData']


In [36]:
# Quantidade de páginas a iterar
qtd_paginas = 100

vetor_anuncios = []

scraper = cloudscraper.create_scraper()

for i in range(1, qtd_paginas + 1):
    try:
        r = scraper.get(f'https://www.olx.com.br/imoveis/venda/apartamentos/estado-pe/grande-recife/recife?o={i}', verify=certifi.where())

        soup = BeautifulSoup(r.text, 'html.parser')
        script_data = soup.find('script', {'id': '__NEXT_DATA__'}).string

        if not script_data:
            print(f"Não foram encontrados dados na página {i}")
            continue

        html = json.loads(script_data)

        dados = html.get('props', {}).get('pageProps', {}).get('ads', [])

        if not dados:
            print(f"Não foram encontrados dados na página {i}")
            continue

        for anuncio in dados:
            dados_anuncio = {
                'Titulo Imovel': anuncio.get('title'),
                'Valor': anuncio.get('price'),
                'Link': anuncio.get('url'),
                'Localizacao': anuncio.get('location'),
                'Bairro': anuncio.get('locationDetails', {}).get('neighbourhood', 'N/A'),
                'Propriedade': anuncio.get('properties'),
                'Adicionais':  anuncio.get('properties')
                }
            
            #verificando títulos inválidos (vazios)
            if dados_anuncio['Titulo Imovel'] is not None:
                vetor_anuncios.append(dados_anuncio)
    
    except Exception as e:
        print(f"Erro na página {i}. Código: {e}")

print("Quantidade de anúncios coletados:", len(vetor_anuncios))


Erro na página 3. Código: 'NoneType' object has no attribute 'string'
Erro na página 9. Código: 'NoneType' object has no attribute 'string'
Erro na página 13. Código: 'NoneType' object has no attribute 'string'
Erro na página 33. Código: 'NoneType' object has no attribute 'string'
Quantidade de anúncios coletados: 4851


In [37]:
df = pd.DataFrame(vetor_anuncios)

In [39]:
# Função para extrair dados da coluna Propriedade com tratamento para None
def extrair_info(properties_list, info_name):
    if properties_list is None:
        return 'N/A'
    for prop in properties_list:
        if prop['name'] == info_name:
            return prop['value']
    return 'N/A'

# Aplicar a função para criar novas colunas
df['Quartos'] = df['Propriedade'].apply(lambda x: extrair_info(x, 'rooms'))
df['Banheiros'] = df['Propriedade'].apply(lambda x: extrair_info(x, 'bathrooms'))
df['Área útil'] = df['Propriedade'].apply(lambda x: extrair_info(x, 'size'))
df['Valor condomínio'] = df['Propriedade'].apply(lambda x: extrair_info(x, 'condominio'))
df['Vagas garagem'] = df['Propriedade'].apply(lambda x: extrair_info(x, 'garage_spaces'))


df_imoveis = df.drop(columns=['Propriedade', 'Adicionais'])

print(df_imoveis.head(5))

                                       Titulo Imovel       Valor  \
0            Candeias Prince - 2102 B - Proprietário  R$ 690.000   
1                         Vende-se Flat em Muro Alto  R$ 510.000   
2  ILHA DE GHARBI - RUA DA HORA ESPINHEIRO. APTO....  R$ 550.000   
3                          Apartamento Parque Prince  R$ 415.000   
4  Imóvel para venda com 18 metros quadrados com ...  R$ 320.000   

                                                Link         Localizacao  \
0  https://pe.olx.com.br/grande-recife/imoveis/ca...  Recife, Boa Viagem   
1  https://pe.olx.com.br/grande-recife/imoveis/ve...    Recife, Madalena   
2  https://pe.olx.com.br/grande-recife/imoveis/il...  Recife, Espinheiro   
3  https://pe.olx.com.br/grande-recife/imoveis/ap...    Recife, Cordeiro   
4  https://pe.olx.com.br/grande-recife/imoveis/im...  Recife, Boa Viagem   

       Bairro Quartos Banheiros Área útil Valor condomínio Vagas garagem  
0  Boa Viagem       3         2      67m²              N/A 

In [40]:
df_imoveis.to_excel('imoveis_recife.xlsx', index=False)

----